# Eden AI

Eden AI is an AI consulting company that was founded to use its resources to empower people and create impactful products that use AI to improve the quality of life for individuals, businesses and societies at large.

This example goes over how to use LangChain to interact with Eden AI models

-----------------------------------------------------------------------------------


Accessing the EDENAI's API requires an API key, 

which you can get by creating an account https://app.edenai.run/user/register  and heading here https://app.edenai.run/admin/account/settings

Once we have a key we'll want to set it as an environment variable by running:



In [ ]:
export EDENAI_API_KEY="..."

If you'd prefer not to set an environment variable you can pass the key in directly via the edenai_api_key named parameter

 when initiating the EdenAI LLM class:



In [3]:
from langchain.llms import EdenAI

In [14]:
llm = EdenAI(edenai_api_key="...",provider="openai", params={"temperature" : 0.2,"max_tokens" : 250})

## Calling a model


The EdenAI API brings together various providers, each offering multiple models.

To access a specific model, you can simply use the "settings" when calling.

For instance, let's explore the models provided by OpenAI, such as GPT3.5 

### text generation

In [11]:
from langchain import PromptTemplate, LLMChain
llm=EdenAI(feature="text",provider="openai", params={"temperature" : 0.2,"max_tokens" : 250})

prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""

llm(prompt,settings={'openai' : 'text-davinci-003'})

" No, a dog cannot drive a car.\n\nReasoning: \n\n1. A dog does not have the physical capability to operate a car. \n2. A dog does not have the cognitive ability to understand the rules of the road and the mechanics of driving. \n3. A dog does not have a driver's license, which is a legal requirement to operate a motor vehicle. \n\nTherefore, a dog cannot drive a car."

### image generation

In [2]:
import base64
from io import BytesIO
from PIL import Image
import json
def print_base64_image(base64_string):
    # Decode the base64 string into binary data
    decoded_data = base64.b64decode(base64_string)

    # Create an in-memory stream to read the binary data
    image_stream = BytesIO(decoded_data)

    # Open the image using PIL
    image = Image.open(image_stream)

    # Display the image
    image.show()

In [3]:
text2image = EdenAI(
    feature="image" ,
    provider= "openai",
    params={
    "resolution" : "512x512"
    }
)

In [4]:
image_output = text2image("A cat riding a motorcycle by Picasso")

In [5]:
print_base64_image(image_output)

### text generation with callback

In [6]:
from langchain.llms import EdenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = EdenAI(
    callbacks=[StreamingStdOutCallbackHandler()],
    feature="text",provider="openai", params={"temperature" : 0.2,"max_tokens" : 250}
)
prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""
print(llm(prompt))

 No, a dog cannot drive a car.

Reasoning:

1. Driving a car requires a driver's license, which only humans can obtain. 
2. Driving a car requires the ability to understand and follow traffic laws, which only humans can do. 
3. Driving a car requires the ability to operate the car's controls, which only humans can do. 
4. Therefore, a dog cannot drive a car.


## Chaining Calls

In [7]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [8]:
llm = EdenAI(
feature="text" ,provider="openai" , params={"temperature" : 0.2,"max_tokens" : 250}
)
text2image = EdenAI(
feature="image" ,provider="openai", params={"resolution" : "512x512"}
)

In [9]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [10]:
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a description of a logo for this company: {company_name}, the logo should not contain text at all ",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [11]:
third_prompt = PromptTemplate(
    input_variables=["company_logo_description"],
    template="{company_logo_description}",
)
chain_three = LLMChain(llm=text2image, prompt=third_prompt)

In [12]:
# Run the chain specifying only the input variable for the first chain.
overall_chain = SimpleSequentialChain(
    chains=[chain, chain_two, chain_three],verbose=True
)
output = overall_chain.run("hats")




> Entering new SimpleSequentialChain chain...




Headwear Haven
. It should be a simple and modern design that conveys the idea of a safe and comfortable place to purchase headwear. The logo should feature a stylized image of a hat, with a warm and inviting color palette. The design should be eye-catching and memorable, and should evoke a sense of trust and reliability.
[1;...

> Finished chain.


In [13]:
#print the image
print_base64_image(output)